In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import json
import requests
from urllib.request import urlopen
from zipfile import ZipFile
from io import BytesIO
import fiona
from shapely.geometry import Point

In [2]:
from methods import read_shst_extract
from methods import link_df_to_geojson
from methods import point_df_to_geojson

In [3]:
data_interim_dir = "../../data/interim/"

# Prepare tomtom for conflation

In [4]:
# loading tomtom data source

gdb_file = "Z:/Data/Users/Sijia/MTC/tomtom/Network_region.gdb"

# Get all the layers from the .gdb file 

layers = fiona.listlayers(gdb_file)

# street link layer

tomtom_raw_gdf = gpd.read_file(gdb_file, layer = 'mn_nw_BayArea')

# convert to ESPG lat-lon

tomtom_raw_gdf = tomtom_raw_gdf.to_crs({'init': 'epsg:4326'})
tomtom_raw_gdf.crs

{'init': 'epsg:4326'}

In [5]:
tomtom_raw_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 602060 entries, 0 to 602059
Data columns (total 53 columns):
ID              602060 non-null float64
FEATTYP         602060 non-null int64
FT              602060 non-null int64
F_JNCTID        602060 non-null float64
F_JNCTTYP       602060 non-null int64
T_JNCTID        602060 non-null float64
T_JNCTTYP       602060 non-null int64
PJ              602060 non-null int64
METERS          602060 non-null float64
FRC             602060 non-null int64
NETCLASS        602060 non-null int64
NETBCLASS       602060 non-null int64
NET2CLASS       602060 non-null int64
NAME            602060 non-null object
NAMELC          602060 non-null object
SOL             602060 non-null int64
NAMETYP         602060 non-null int64
CHARGE          602060 non-null object
SHIELDNUM       602060 non-null object
RTETYP          602060 non-null int64
RTEDIR          602060 non-null object
RTEDIRVD        602060 non-null object
PROCSTAT        602060 non-null

In [39]:
# There is no existing unique tomtom handle for Bay Area, thus we need to create unique handle

In [6]:
# total # tomtom links

tomtom_raw_gdf.shape

(602060, 53)

In [7]:
# unique tomtom handles

len(tomtom_raw_gdf.groupby(["ID", "F_JNCTID", "T_JNCTID"]).count())

601846

In [8]:
# generating unique handle for tomtom

tomtom_raw_gdf["tomtom_link_id"] = range(1, len(tomtom_raw_gdf)+1)

In [9]:
for line in tomtom_raw_gdf[tomtom_raw_gdf.tomtom_link_id == 97874]["geometry"].iloc[0]:
     print(line)

LINESTRING (-122.4323109994277 37.76753700037561, -122.4334150001658 37.76747600031828)


# Partition tomtom by county for shst match

In [44]:
for i in range(14):
    boundary_gdf = gpd.read_file("../../data/external/county_boundaries/boundary_"+str(i+1)+".geojson")
    
    sub_tomtom_gdf = tomtom_raw_gdf[tomtom_raw_gdf.intersects(boundary_gdf.geometry.unary_union)].copy()
    
    sub_tomtom_gdf[["tomtom_link_id", "geometry"]].to_file("../../data/external/tomtom/tomtom"+str(i+1)+".in.geojson",
                                                driver = "GeoJSON")

In [ ]:
tomtom_raw_gdf.head(3)

In [ ]:
tomtom_raw_gdf[~(tomtom_raw_gdf.geom_type == 'MultiLineString')]

In [66]:
tomtom_raw_gdf.crs

{'init': 'epsg:4326'}

# Prepare TM2 for conflation

In [44]:
# loading TM2 non Marin version data source
# http://bayareametro.github.io/travel-model-two/input/#roadway-network

tm2_link_file = "../../data/external/TM2_nonMarin/mtc_final_network_base.shp"

# street link layer

tm2_link_gdf = gpd.read_file(tm2_link_file)


In [45]:
tm2_link_gdf.crs = {"init" : "esri:102646"}

# convert to ESPG lat-lon

tm2_link_gdf = tm2_link_gdf.to_crs({'init': 'epsg:4326'})
tm2_link_gdf.crs

{'init': 'epsg:4326'}

In [46]:
tm2_link_gdf.shape

(1405145, 41)

In [47]:
tm2_link_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1405145 entries, 0 to 1405144
Data columns (total 41 columns):
A              1405145 non-null int64
B              1405145 non-null int64
NUMLANES       1405145 non-null int64
F_JNCTID       1405145 non-null int64
T_JNCTID       1405145 non-null int64
FRC            1405145 non-null int64
NAME           741995 non-null object
FREEWAY        1405145 non-null int64
TOLLRD         291 non-null object
ONEWAY         129292 non-null object
KPH            1405145 non-null int64
MINUTES        1405145 non-null float64
CARRIAGE       100 non-null object
LANES          1405145 non-null int64
RAMP           1405145 non-null int64
SPEEDCAT       1405145 non-null int64
FEET           1405145 non-null float64
ASSIGNABLE     1405145 non-null int64
CNTYPE         1405113 non-null object
TRANSIT        1405145 non-null int64
HIGHWAYT       223551 non-null object
B_CLASS        1405145 non-null int64
GRADE_CAT      1405145 non-null int64
PED_FL

In [48]:
tm2_link_gdf.NUMLANES.value_counts()

1     722068
0     596627
2      51515
3      23005
4       8390
5       2839
6        470
7        166
9         50
8         13
16         2
Name: NUMLANES, dtype: int64

In [49]:
tm2_link_gdf.LANES.value_counts()

0     1253742
2       70384
3       29055
1       17060
4       16329
7        9170
5        8256
6        1071
9          50
8          26
16          2
Name: LANES, dtype: int64

In [13]:
tm2_link_gdf[(tm2_link_gdf.LANES != tm2_link_gdf.NUMLANES) & (tm2_link_gdf.LANES>0)]

,A,B,NUMLANES,F_JNCTID,T_JNCTID,FRC,NAME,FREEWAY,TOLLRD,ONEWAY,...,TAP_DRIVE,FT,FFS,USECLASS,TOLLBOOTH,DANGLING,HASTRANSIT,DELETE,TOLLSEG,geometry
179791,1000500,1004570,1,0,0,4,San Jose Ave,0,None,None,...,0,7,30.0,0,0,0,0,0,0,LINESTRING (-122.4404539929942 37.728443009645...
179793,1000500,1007363,1,0,0,4,San Jose Ave,0,None,None,...,0,7,30.0,0,0,0,0,0,0,LINESTRING (-122.4404539929942 37.728443009645...
179819,1000507,1003766,1,0,0,5,Silver Ave,0,None,None,...,0,7,25.0,0,0,0,21,0,0,LINESTRING (-122.4264260108296 37.728618990649...
179821,1000507,1012507,1,0,0,5,Silver Ave,0,None,None,...,0,7,25.0,0,0,0,2,0,0,LINESTRING (-122.4264260108296 37.728618990649...
179899,1000531,1003867,2,0,0,4,Fulton St,0,None,None,...,0,7,30.0,0,0,0,3,0,0,LINESTRING (-122.4816199876643 37.772668999510...
179900,1000531,1007467,2,0,0,4,Fulton St,0,None,None,...,0,7,30.0,0,0,0,3,0,0,LINESTRING (-122.4816199876643 37.772668999510...
179907,1000532,1011072,1,0,0,5,Cortland Ave,0,None,None,...,0,7,25.0,0,0,0,4,0,0,LINESTRING (-122.4197240140713 37.739758991057...
179908,1000532,1013730,1,0,0,5,Cortland Ave,0,None,None,...,0,7,25.0,0,0,0,4,0,0,LINESTRING (-122.4197240140713 37.739758991057...
179928,1000538,1005887,1,0,0,5,Mission St,0,None,None,...,0,7,25.0,0,0,0,5,0,0,LINESTRING (-122.399749687076 37.7880426906578...
179929,1000538,1031004,1,0,0,5,Mission St,0,None,None,...,0,7,25.0,0,0,0,6,0,0,LINESTRING (-122.399749687076 37.7880426906578...


In [50]:
tm2_link_gdf.CNTYPE.value_counts(dropna = False)

TANA     807224
MAZ      252437
PED      221905
TAZ       63182
TAP       43512
USE        8981
BIKE       6678
LRAIL       864
CRAIL       144
HRAIL        90
FERRY        52
EXT          44
NaN          32
Name: CNTYPE, dtype: int64

In [139]:
tm2_link_gdf[tm2_link_gdf.CNTYPE.isnull()]

,A,B,NUMLANES,F_JNCTID,T_JNCTID,FRC,NAME,FREEWAY,TOLLRD,ONEWAY,...,TAP_DRIVE,FT,FFS,USECLASS,TOLLBOOTH,DANGLING,HASTRANSIT,DELETE,TOLLSEG,geometry
267141,1031128,1029524,0,0,0,0,None,0,None,None,...,0,0,0.0,0,0,0,0,0,0,LINESTRING (-122.4956800080602 37.759029007638...
276860,1502324,1509295,0,0,0,0,None,0,None,None,...,0,0,0.0,0,0,0,0,1,0,LINESTRING (-122.2528569885394 37.527925005355...
320209,1517754,1509295,0,0,0,0,None,0,None,None,...,0,0,0.0,0,0,0,0,0,0,LINESTRING (-122.2527820117052 37.527870010715...
400056,2004231,2103238,0,0,0,0,None,0,None,None,...,0,0,0.0,0,0,0,0,1,0,LINESTRING (-122.0884999919633 37.423662372169...
558602,2067435,2069921,0,0,0,0,None,0,None,None,...,0,0,0.0,0,0,0,0,0,0,LINESTRING (-122.1664369966959 37.433450006054...
567528,2070694,2065085,0,0,0,0,None,0,None,None,...,0,0,0.0,0,0,0,0,0,0,LINESTRING (-122.1746359905483 37.440286001030...
573584,2072913,2068599,0,0,0,0,None,0,None,None,...,0,0,0.0,0,0,0,0,0,0,LINESTRING (-122.0905280134502 37.424397011729...
633503,2100138,2065085,0,0,0,0,None,0,None,None,...,0,0,0.0,0,0,0,0,1,0,LINESTRING (-122.1755164099981 37.439499004042...
684202,2505226,2567475,0,0,0,0,None,0,None,None,...,0,0,0.0,0,0,0,0,0,0,LINESTRING (-122.2694512412447 37.823925618919...
703975,2513867,2582543,0,0,0,0,None,0,None,None,...,0,0,0.0,0,0,0,0,0,0,LINESTRING (-122.2761119958936 37.779635987576...


In [9]:
tm2_link_gdf[(tm2_link_gdf.A == 3027596) & (tm2_link_gdf.B == 3028181)]

,A,B,NUMLANES,F_JNCTID,T_JNCTID,FRC,NAME,FREEWAY,TOLLRD,ONEWAY,...,TAP_DRIVE,FT,FFS,USECLASS,TOLLBOOTH,DANGLING,HASTRANSIT,DELETE,TOLLSEG,geometry
982140,3027596,3028181,1,0,0,6,Bettencourt Dr,0,None,None,...,0,4,20.0,0,0,0,0,1,0,LINESTRING (-121.9424930081923 37.951476007482...


In [136]:
tm2_link_gdf[tm2_link_gdf.CNTYPE == "TAP"]

,A,B,NUMLANES,F_JNCTID,T_JNCTID,FRC,NAME,FREEWAY,TOLLRD,ONEWAY,...,TAP_DRIVE,FT,FFS,USECLASS,TOLLBOOTH,DANGLING,HASTRANSIT,DELETE,TOLLSEG,geometry
14510,90001,1007968,0,0,0,0,None,0,None,None,...,0,6,25.0,0,0,0,0,0,0,LINESTRING (-122.4850680767219 37.785822429083...
14511,90001,1015082,0,0,0,0,None,0,None,None,...,0,6,25.0,0,0,0,0,0,0,LINESTRING (-122.4850680767219 37.785822429083...
14512,90001,1029156,0,0,0,0,None,0,None,None,...,0,6,25.0,0,0,0,0,0,0,LINESTRING (-122.4850680767219 37.785822429083...
14513,90002,1010258,0,0,0,0,None,0,None,None,...,0,6,25.0,0,0,0,0,0,0,LINESTRING (-122.4464040516504 37.775892401385...
14514,90002,1031191,0,0,0,0,None,0,None,None,...,0,6,25.0,0,0,0,0,0,0,LINESTRING (-122.4464040516504 37.775892401385...
14515,90003,1003729,0,0,0,0,None,0,None,None,...,0,6,25.0,0,0,0,0,0,0,LINESTRING (-122.3967519688971 37.795590370765...
14516,90003,1014366,0,0,0,0,None,0,None,None,...,0,6,25.0,0,0,0,0,0,0,LINESTRING (-122.3967519688971 37.795590370765...
14517,90004,1029621,0,0,0,0,None,0,None,None,...,0,6,25.0,0,0,0,0,0,0,LINESTRING (-122.47627410276 37.75431540940309...
14518,90004,1029799,0,0,0,0,None,0,None,None,...,0,6,25.0,0,0,0,0,0,0,LINESTRING (-122.47627410276 37.75431540940309...
14519,90004,1030233,0,0,0,0,None,0,None,None,...,0,6,25.0,0,0,0,0,0,0,LINESTRING (-122.47627410276 37.75431540940309...


In [20]:
tm2_link_gdf.CNTYPE.value_counts()

TANA     807224
MAZ      252437
PED      221905
TAZ       63182
TAP       43512
USE        8981
BIKE       6678
LRAIL       864
CRAIL       144
HRAIL        90
FERRY        52
EXT          44
Name: CNTYPE, dtype: int64

In [51]:
tm2_link_roadway_gdf = tm2_link_gdf[tm2_link_gdf.CNTYPE.isin(["BIKE", "PED", "TANA"])].copy()

In [52]:
tm2_link_roadway_gdf.shape

(1035807, 41)

In [53]:
tm2_link_roadway_gdf.groupby(["A", "B"]).count().shape

(1035807, 39)

In [80]:
tm2_link_roadway_gdf.head(3)

,A,B,NUMLANES,F_JNCTID,T_JNCTID,FRC,NAME,FREEWAY,TOLLRD,ONEWAY,...,TAP_DRIVE,FT,FFS,USECLASS,TOLLBOOTH,DANGLING,HASTRANSIT,DELETE,TOLLSEG,geometry
178886,1000000,1001408,0,0,0,0,None,0,None,None,...,0,0,0.0,0,0,0,0,1,0,LINESTRING (-122.4631000108774 37.772600013153...
178887,1000000,1032451,0,0,0,0,None,0,None,None,...,0,0,0.0,0,0,0,0,1,0,LINESTRING (-122.4631000108774 37.772600013153...
178888,1000001,1023686,0,0,0,0,None,0,None,None,...,0,0,0.0,0,0,0,0,1,0,LINESTRING (-122.3923000045329 37.775399991779...


# Partition TM2 Non Marin for shst Match

In [82]:
for i in range(14):
    boundary_gdf = gpd.read_file("../../data/external/county_boundaries/boundary_"+str(i+1)+".geojson")
    
    sub_gdf = tm2_link_roadway_gdf[tm2_link_roadway_gdf.intersects(boundary_gdf.geometry.unary_union)].copy()
    
    sub_gdf[["A", "B", "geometry"]].to_file("../../data/external/TM2_nonMarin/tm2nonMarin_"+str(i+1)+".in.geojson",
                                                driver = "GeoJSON")

In [532]:
for i in [4]:
    boundary_gdf = gpd.read_file("../../data/external/county_boundaries/boundary_"+str(4)+".geojson")
    
    sub_gdf = tm2_link_roadway_gdf[tm2_link_roadway_gdf.intersects(boundary_gdf.geometry.unary_union)].copy()
    
    sub_gdf[["A", "B", "geometry", "ASSIGNABLE"]].to_file("../../data/external/TM2_nonMarin/tm2nonMarin_"+str(4)+"assignable.in.geojson",
                                                driver = "GeoJSON")

# Prepare TM2 Marin for conflation

In [54]:
# loading TM2 Marin version data source
# http://bayareametro.github.io/travel-model-two/input/#roadway-network

tm2_marin_link_file = "../../data/external/TM2_Marin/mtc_final_network_base.shp"

# street link layer

tm2_marin_link_gdf = gpd.read_file(tm2_marin_link_file)

In [55]:
tm2_marin_link_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1405258 entries, 0 to 1405257
Data columns (total 47 columns):
A              1405258 non-null int64
B              1405258 non-null int64
F_JNCTID       1405258 non-null int64
T_JNCTID       1405258 non-null int64
FRC            1405258 non-null int64
NAME           741998 non-null object
FREEWAY        1405258 non-null int64
TOLLRD         291 non-null object
ONEWAY         129297 non-null object
KPH            1405258 non-null int64
MINUTES        1405258 non-null float64
CARRIAGE       100 non-null object
LANES          1405258 non-null int64
RAMP           1405258 non-null int64
SPEEDCAT       1405258 non-null int64
FEET           1405258 non-null float64
ASSIGNABLE     1405258 non-null int64
CNTYPE         1405200 non-null object
TRANSIT        1405258 non-null int64
HIGHWAYT       223551 non-null object
B_CLASS        1405258 non-null int64
GRADE_CAT      1405258 non-null int64
PED_FLAG       133180 non-null object
BIKEPE

In [56]:
tm2_marin_link_gdf.LANES.value_counts()

0     1254103
2       70434
3       29085
1       16905
4       16360
7        8964
5        8257
6        1072
9          50
8          26
16          2
Name: LANES, dtype: int64

In [57]:
tm2_marin_link_gdf.NUMLANES.value_counts()

1     720966
0     596813
2      52147
3      23343
4       8431
5       2840
6        469
7        184
9         50
8         13
16         2
Name: NUMLANES, dtype: int64

In [58]:
tm2_marin_link_gdf.CNTYPE.value_counts()

TANA     807115
MAZ      252435
PED      221905
TAZ       63182
TAP       43894
USE        8797
BIKE       6678
LRAIL       864
CRAIL       144
HRAIL        90
FERRY        52
EXT          44
Name: CNTYPE, dtype: int64

In [59]:
tm2_marin_link_gdf.crs = {"init" : "esri:102646"}

# convert to ESPG lat-lon

tm2_marin_link_gdf = tm2_marin_link_gdf.to_crs({'init': 'epsg:4326'})
tm2_marin_link_gdf.crs

{'init': 'epsg:4326'}

In [60]:
tm2_marin_link_roadway_gdf = tm2_marin_link_gdf[tm2_marin_link_gdf.CNTYPE.isin(["BIKE", "PED", "TANA"])].copy()

In [61]:
tm2_marin_link_roadway_gdf.shape

(1035698, 47)

In [62]:
tm2_marin_link_roadway_gdf.groupby(["A", "B"]).count().shape

(1035698, 45)

In [63]:
tm2_link_roadway_gdf.FRC.value_counts()

6    326320
7    270378
0    235558
4     85342
5     81408
3     22340
1      8590
8      4621
2      1250
Name: FRC, dtype: int64

In [64]:
tm2_marin_link_roadway_gdf.FRC.value_counts()

6    326322
7    270378
0    235461
4     85342
5     81408
3     22341
1      8575
8      4621
2      1250
Name: FRC, dtype: int64

# Partition TM2 Marin for shst Match 

In [92]:
for i in range(14):
    boundary_gdf = gpd.read_file("../../data/external/county_boundaries/boundary_"+str(i+1)+".geojson")
    
    sub_gdf = tm2_marin_link_roadway_gdf[tm2_marin_link_roadway_gdf.intersects(boundary_gdf.geometry.unary_union)].copy()
    
    sub_gdf[["A", "B", "geometry"]].to_file("../../data/external/TM2_Marin/tm2Marin_"+str(i+1)+".in.geojson",
                                                driver = "GeoJSON")

# Prepared SFCTA true shape file for conflation

In [65]:
sfcta_file = "../../data/external/stclines/stclines.shp"

sfcta_gdf = gpd.read_file(sfcta_file)

In [66]:
sfcta_gdf.crs = {"init" : "epsg:2227"}

In [67]:
# convert to ESPG lat-lon

sfcta_gdf = sfcta_gdf.to_crs({'init': 'epsg:4326'})
sfcta_gdf.crs

{'init': 'epsg:4326'}

In [68]:
sfcta_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 16165 entries, 0 to 16164
Data columns (total 23 columns):
OBJECTID      16165 non-null int64
CNN           16165 non-null float64
STREET        16145 non-null object
ST_TYPE       15713 non-null object
LF_FADD       16165 non-null float64
LF_TOADD      16165 non-null float64
RT_FADD       16165 non-null float64
RT_TOADD      16165 non-null float64
F_NODE_CNN    16165 non-null int64
T_NODE_CNN    16165 non-null int64
ZIP_CODE      16071 non-null object
DISTRICT      16159 non-null object
ACCEPTED      16159 non-null object
JURISDICTI    12980 non-null object
NHOOD         16159 non-null object
LAYER         16159 non-null object
CNNTEXT       16148 non-null object
STREETNAME    16161 non-null object
A             16165 non-null int64
B             16165 non-null int64
AB            16164 non-null object
MEDIANDIV     16165 non-null int64
geometry      16165 non-null object
dtypes: float64(5), int64(6), object(12)
memory usage: 2

In [69]:
# there's no unique id for sfcta true shape file, not even combinations of A/B

sfcta_gdf["sfcta_trueshape_id"] = range(1, len(sfcta_gdf) + 1)

In [70]:
sfcta_gdf.drop_duplicates(subset = sfcta_gdf.drop(["geometry"], axis=1).columns)

,OBJECTID,CNN,STREET,ST_TYPE,LF_FADD,LF_TOADD,RT_FADD,RT_TOADD,F_NODE_CNN,T_NODE_CNN,...,NHOOD,LAYER,CNNTEXT,STREETNAME,A,B,AB,MEDIANDIV,geometry,sfcta_trueshape_id
0,1,7520002.0,JESSIE,ST,351.0,399.0,350.0,398.0,54081000,24884000,...,Financial District South,STREETS,7520002,JESSIE ST,54081,24884,54081 24884,0,LINESTRING (-122.4065825247182 37.783499222590...,1
1,2,7520001.0,JESSIE,ST,301.0,349.0,300.0,348.0,24635000,54080000,...,Financial District South,STREETS,7520001,JESSIE ST,24635,54080,24635 54080,0,LINESTRING (-122.4047950624642 37.784908299223...,2
2,3,3186101.0,ROBERT C LEVY,TUNL,0.0,0.0,0.0,0.0,25111000,25308000,...,Nob Hill,STREETS,3186101,ROBERT C LEVY TUNL,25111,25308,25111 25308,0,LINESTRING (-122.4102635257008 37.797420184978...,3
3,4,3186201.0,ROBERT C LEVY,TUNL,0.0,0.0,0.0,0.0,25111000,25308000,...,Nob Hill,STREETS,3186201,ROBERT C LEVY TUNL,25111,25308,25111 25308,0,LINESTRING (-122.4102635257008 37.797420184978...,4
4,5,5754001.0,FRANCISCO,ST,101.0,175.0,100.0,174.0,25385000,50352000,...,Telegraph Hill,UPROW,5754001,FRANCISCO ST,25385,50352,25385 50352,0,LINESTRING (-122.4069566340834 37.805494468097...,5
5,6,5491001.0,FILBERT,ST,211.0,219.0,216.0,222.0,50354000,24865000,...,Telegraph Hill,UPROW,5491001,FILBERT ST,50354,24865,50354 24865,0,LINESTRING (-122.4033973604645 37.802113386380...,6
6,7,13087001.0,VALLEJO,ST,401.0,433.0,400.0,432.0,24841000,50210000,...,Telegraph Hill,PAPER,13087001,VALLEJO ST,24841,50210,24841 50210,0,LINESTRING (-122.4040531316579 37.799172217316...,7
7,8,3126001.0,BRIGHT,ST,51.0,99.0,50.0,98.0,54078000,54079000,...,Ingleside Heights,Paper_fwys,3126001,BRIGHT ST,54078,54079,54078 54079,0,LINESTRING (-122.4634754559928 37.711861596185...,8
8,9,5769000.0,FRANCISCO,ST,1001.0,1099.0,1000.0,1098.0,25581000,26702000,...,Russian Hill,STREETS,5769000,FRANCISCO ST,25581,26702,25581 26702,0,LINESTRING (-122.4216263341576 37.803502957971...,9
9,10,8155001.0,LARKIN,ST,2801.0,2849.0,2800.0,2848.0,25581000,50368000,...,Russian Hill,PAPER,8155001,LARKIN ST,25581,50368,25581 50368,0,LINESTRING (-122.4216263341576 37.803502957971...,10


In [71]:
sfcta_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 16165 entries, 0 to 16164
Data columns (total 24 columns):
OBJECTID              16165 non-null int64
CNN                   16165 non-null float64
STREET                16145 non-null object
ST_TYPE               15713 non-null object
LF_FADD               16165 non-null float64
LF_TOADD              16165 non-null float64
RT_FADD               16165 non-null float64
RT_TOADD              16165 non-null float64
F_NODE_CNN            16165 non-null int64
T_NODE_CNN            16165 non-null int64
ZIP_CODE              16071 non-null object
DISTRICT              16159 non-null object
ACCEPTED              16159 non-null object
JURISDICTI            12980 non-null object
NHOOD                 16159 non-null object
LAYER                 16159 non-null object
CNNTEXT               16148 non-null object
STREETNAME            16161 non-null object
A                     16165 non-null int64
B                     16165 non-null int64
AB 

In [72]:
sfcta_gdf.OBJECTID.nunique()

16091

In [42]:
sfcta_gdf.groupby(["OBJECTID", "AB"]).count().shape

(16117, 22)

# Partition SFCTA for conflation

In [167]:
# only conflatio SF county from SFCTA

sfcta_gdf[["sfcta_trueshape_id", "geometry"]].to_file("../../data/external/sfclines/sfcta.in.geojson",
                                                driver = "GeoJSON")

# Prepare SFCTA Stick Network for conflation

In [73]:
sfcta_stick_file = "../../data/external/sfcta/SanFrancisco_links.shp"

sfcta_stick_gdf = gpd.read_file(sfcta_stick_file)

In [74]:
sfcta_stick_gdf.crs = {"init" : "epsg:2227"}

In [75]:
# convert to ESPG lat-lon

sfcta_stick_gdf = sfcta_stick_gdf.to_crs({'init': 'epsg:4326'})
sfcta_stick_gdf.crs

{'init': 'epsg:4326'}

In [46]:
sfcta_stick_gdf

,A,B,TOLL,USE,CAP,AT,FT,STREETNAME,TYPE,MTYPE,...,TSIN,BIKE_CLASS,PER_RISE,ONEWAY,PROJ,DTA_EDIT_FL,TOLLTIME,PHASE,ACTION,geometry
0,1,20368,0,1,2000,3,6,None,None,SF,...,T,0,0.0,0,None,0,0.0,0,None,LINESTRING (-122.413689287528 37.7094935033756...
1,1,20373,0,1,2000,3,6,None,None,SF,...,T,0,0.0,0,None,0,0.0,0,None,LINESTRING (-122.413689287528 37.7094935033756...
2,1,20375,0,1,2000,3,6,None,None,SF,...,T,0,0.0,0,None,0,0.0,0,None,LINESTRING (-122.413689287528 37.7094935033756...
3,1,20408,0,1,2000,3,6,None,None,SF,...,T,0,0.0,0,None,0,0.0,0,None,LINESTRING (-122.413689287528 37.7094935033756...
4,1,33349,0,1,2000,3,6,None,None,SF,...,T,0,0.0,0,None,0,0.0,0,None,LINESTRING (-122.413689287528 37.7094935033756...
5,2,22479,0,1,2000,2,6,None,None,SF,...,T,0,0.0,0,None,0,0.0,0,None,LINESTRING (-122.4636867689285 37.709210357198...
6,2,22482,0,1,2000,2,6,None,None,SF,...,T,0,0.0,0,None,0,0.0,0,None,LINESTRING (-122.4636867689285 37.709210357198...
7,2,22483,0,1,2000,2,6,None,None,SF,...,T,0,0.0,0,None,0,0.0,0,None,LINESTRING (-122.4636867689285 37.709210357198...
8,2,22513,0,1,2000,2,6,None,None,SF,...,T,0,0.0,0,None,0,0.0,0,None,LINESTRING (-122.4636867689285 37.709210357198...
9,2,33454,0,1,2000,2,6,None,None,SF,...,T,0,0.0,0,None,0,0.0,0,None,LINESTRING (-122.4636867689285 37.709210357198...


In [47]:
sfcta_stick_gdf.columns

Index(['A', 'B', 'TOLL', 'USE', 'CAP', 'AT', 'FT', 'STREETNAME', 'TYPE',
       'MTYPE', 'SPEED', 'DISTANCE', 'TIME', 'LANE_AM', 'LANE_OP', 'LANE_PM',
       'BUSLANE_AM', 'BUSLANE_OP', 'BUSLANE_PM', 'TOLLAM_DA', 'TOLLAM_SR2',
       'TOLLAM_SR3', 'TOLLPM_DA', 'TOLLPM_SR2', 'TOLLPM_SR3', 'TOLLEA_DA',
       'TOLLEA_SR2', 'TOLLEA_SR3', 'TOLLMD_DA', 'TOLLMD_SR2', 'TOLLMD_SR3',
       'TOLLEV_DA', 'TOLLEV_SR2', 'TOLLEV_SR3', 'VALUETOLL_F', 'PASSTHRU',
       'BUSTPS_AM', 'BUSTPS_OP', 'BUSTPS_PM', 'TSVA', 'TSIN', 'BIKE_CLASS',
       'PER_RISE', 'ONEWAY', 'PROJ', 'DTA_EDIT_FL', 'TOLLTIME', 'PHASE',
       'ACTION', 'geometry'],
      dtype='object')

# Write out SFCTA stick network for conflation

In [385]:
boundary_4_gdf = gpd.read_file("../../data/external/county_boundaries/boundary_4.geojson")

sfcta_stick_roadway_gdf = sfcta_stick_gdf[~(sfcta_stick_gdf.FT == 6)].copy()

sfcta_stick_roadway_gdf[sfcta_stick_roadway_gdf.intersects(boundary_4_gdf.geometry.unary_union)][['A', 'B', "geometry"]].to_file(
    "../../data/external/sfcta/sfcta_in.geojson",
                                            driver="GeoJSON")

# Prepare PEMS for conflation

In [132]:
pems_file = "../../data/external/mtc/pems_period.csv"

pems_df = pd.read_csv(pems_file)

In [198]:
pems_df.columns

Index(['station', 'district', 'route', 'direction', 'type', 'time_period',
       'lanes', 'median_flow', 'avg_flow', 'sd_flow', 'median_speed',
       'avg_speed', 'sd_speed', 'median_occup', 'avg_occup', 'sd_occupancy',
       'days_observed', 'state_pm', 'abs_pm', 'latitude', 'longitude', 'year',
       'geometry'],
      dtype='object')

In [279]:
pems_df.columns

Index(['station', 'district', 'route', 'direction', 'type', 'time_period',
       'lanes', 'median_flow', 'avg_flow', 'sd_flow', 'median_speed',
       'avg_speed', 'sd_speed', 'median_occup', 'avg_occup', 'sd_occupancy',
       'days_observed', 'state_pm', 'abs_pm', 'latitude', 'longitude', 'year',
       'geometry'],
      dtype='object')

In [282]:
pems_df[pems_df.station == 404912]

,station,district,route,direction,type,time_period,lanes,median_flow,avg_flow,sd_flow,...,median_occup,avg_occup,sd_occupancy,days_observed,state_pm,abs_pm,latitude,longitude,year,geometry
58462,404912,4,80,E,ML,AM,5,21710.5,21609.441176,585.715682,...,0.086007,0.086370,0.007972,34,39.0,5.87,37.819788,-122.339284,2015,POINT (-122.339284 37.819788)
58463,404912,4,80,E,ML,EA,5,3655.0,3640.545455,160.732909,...,0.023093,0.026530,0.011436,33,39.0,5.87,37.819788,-122.339284,2015,POINT (-122.339284 37.819788)
58464,404912,4,80,E,ML,EV,5,36117.5,36121.928571,2254.275492,...,0.125401,0.120350,0.025382,28,39.0,5.87,37.819788,-122.339284,2015,POINT (-122.339284 37.819788)
58465,404912,4,80,E,ML,MD,5,31697.0,31727.400000,1833.019886,...,0.100678,0.104338,0.013787,30,39.0,5.87,37.819788,-122.339284,2015,POINT (-122.339284 37.819788)
58466,404912,4,80,E,ML,PM,5,32495.0,32299.516129,725.068037,...,0.136917,0.140920,0.016671,31,39.0,5.87,37.819788,-122.339284,2015,POINT (-122.339284 37.819788)
67803,404912,4,80,E,ML,AM,5,21893.5,21657.166667,1198.052410,...,0.076067,0.078784,0.009290,72,40.0,5.87,37.819788,-122.339284,2016,POINT (-122.339284 37.819788)
67804,404912,4,80,E,ML,EA,5,3734.0,3736.959459,167.699866,...,0.019176,0.019860,0.003401,74,40.0,5.87,37.819788,-122.339284,2016,POINT (-122.339284 37.819788)
67805,404912,4,80,E,ML,EV,5,38699.0,38742.183099,2579.255337,...,0.087891,0.091310,0.014602,71,40.0,5.87,37.819788,-122.339284,2016,POINT (-122.339284 37.819788)
67806,404912,4,80,E,ML,MD,5,32742.0,32568.920635,1425.203147,...,0.095564,0.096204,0.009059,63,40.0,5.87,37.819788,-122.339284,2016,POINT (-122.339284 37.819788)
67807,404912,4,80,E,ML,PM,5,32201.5,32064.352941,1156.708593,...,0.120003,0.123810,0.022957,68,40.0,5.87,37.819788,-122.339284,2016,POINT (-122.339284 37.819788)


In [173]:
pems_df["geometry"] = [Point(xy) for xy in zip(pems_df.longitude, pems_df.latitude)]

pems_gdf = gpd.GeoDataFrame(pems_df)

In [186]:
pems_gdf = pems_gdf[~((pems_gdf.longitude.isnull()) | (pems_gdf.latitude.isnull()))]

In [190]:
# keep unique

pems_gdf.drop_duplicates(subset = ["station", "longitude", "latitude"])[["station", "longitude", "latitude", "geometry"]].to_file("../../data/external/mtc/pems.in.geojson",
                                                driver = "GeoJSON")

# Read network

In [97]:
link_file = data_interim_dir + "step3_join_shst_extraction_with_osm/" + "link.json"
with open(link_file) as f:
    link_json = json.load(f)
link_df = pd.DataFrame(link_json)

shape_gdf = gpd.read_file(data_interim_dir + "step3_join_shst_extraction_with_osm/" 
                          + "shape.geojson")

link_gdf = pd.merge(link_df,
                    shape_gdf[["id", "geometry"]],
                   how = "left",
                   on = "id")

link_gdf = gpd.GeoDataFrame(link_gdf,
                            crs="EPSG:4326")

In [100]:
link_gdf[(link_gdf.u == 4075004794) & (link_gdf.v == 4075001784)]

,access,area,bike_access,bridge,drive_access,est_width,fromIntersectionId,highway,id,junction,...,shstGeometryId,shstReferenceId,toIntersectionId,tunnel,u,v,walk_access,wayId,width,geometry
348788,nan,nan,1,nan,0,nan,f407efc156163c556b516200a966187e,track,b5a13d71332300baf271a3dde50dd06d,nan,...,b5a13d71332300baf271a3dde50dd06d,6233a20ff0d9dc29be597a06ffd19727,7da9cd62c681f3425ba9b2e28d702fc9,nan,4075004794,4075001784,1,405292130,nan,"LINESTRING (-122.2339452 37.4206079, -122.2341..."
1129778,"[nan, nan]","[nan, nan]",1,"[nan, nan]",1,"[nan, nan]",f407efc156163c556b516200a966187e,"[service, track]",b6f6f29894314a39e1b7069bd4f53d96,"[nan, nan]",...,b6f6f29894314a39e1b7069bd4f53d96,4715d32ebbf18e592af26b06ce167ac0,7da9cd62c681f3425ba9b2e28d702fc9,"[nan, nan]",4075004794,4075001784,1,"[405292128, 405292129]","[nan, nan]","LINESTRING (-122.2364996 37.4191189, -122.2365..."


In [101]:
link_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 1705772 entries, 0 to 1705771
Data columns (total 31 columns):
access                object
area                  object
bike_access           int64
bridge                object
drive_access          int64
est_width             object
fromIntersectionId    object
highway               object
id                    object
junction              object
key                   object
landuse               object
lanes                 object
link                  object
maxspeed              object
name                  object
oneWay                object
ref                   object
roadway               object
roundabout            object
service               object
shstGeometryId        object
shstReferenceId       object
toIntersectionId      object
tunnel                object
u                     int64
v                     int64
walk_access           int64
wayId                 object
width                 object
geometry      

# Read TomTom ShSt Match result

In [12]:
# read in tomtom conflation

tomtom_match_gdf = read_shst_extract("D:/MTC/data/interim/" + "tomtom/", "*_tomtom.out.matched.geojson")

tomtom_match_gdf.rename(columns = {"shstFromIntersectionId" : "fromIntersectionId",
                                   "shstToIntersectionId" : "toIntersectionId"},
                       inplace = True)

----------start reading shst extraction data-------------
reading shst extraction data :  D:/MTC/data/interim/tomtom\bike_rules\1_tomtom.out.matched.geojson
reading shst extraction data :  D:/MTC/data/interim/tomtom\bike_rules\2_tomtom.out.matched.geojson
reading shst extraction data :  D:/MTC/data/interim/tomtom\car_rules\10_tomtom.out.matched.geojson
reading shst extraction data :  D:/MTC/data/interim/tomtom\car_rules\11_tomtom.out.matched.geojson
reading shst extraction data :  D:/MTC/data/interim/tomtom\car_rules\12_tomtom.out.matched.geojson
reading shst extraction data :  D:/MTC/data/interim/tomtom\car_rules\13_tomtom.out.matched.geojson
reading shst extraction data :  D:/MTC/data/interim/tomtom\car_rules\14_tomtom.out.matched.geojson
reading shst extraction data :  D:/MTC/data/interim/tomtom\car_rules\1_tomtom.out.matched.geojson
reading shst extraction data :  D:/MTC/data/interim/tomtom\car_rules\2_tomtom.out.matched.geojson
reading shst extraction data :  D:/MTC/data/interim/t

In [13]:
tomtom_match_gdf

,shstReferenceId,shstGeometryId,fromIntersectionId,toIntersectionId,gisReferenceId,gisGeometryId,gisTotalSegments,gisSegmentIndex,gisFromIntersectionId,gisToIntersectionId,startSideOfStreet,endSideOfStreet,sideOfStreet,score,matchType,pp_tomtom_link_id,geometry,source
0,340aca5eed3be40e615422d39544d223,52a5e0b4800e7283accda73c74108762,8711d153fefd546c2845baeb0f67bd68,ce8aa7e55b3c5f951154f22d1fb5efd0,35317b9b780d7f5843d00a207b13c092,d0b88c4ea04cca9bf9835f85df44ed32,2,1,91cf2a0dbffb2e7f14c03e72e704d3e5,a3ac6e509c62ac3d7796c8d8b75f72b2,right,left,unknown,4.68,hmm,337110,LINESTRING (-121.9121105815505 38.014193818053...,D:/MTC/data/interim/tomtom\bike_rules\1_tomtom...
1,4b1dd93afef058cf6b751dd55aa40031,2eba19c9a54083c15f8fd70e025559b4,ce8aa7e55b3c5f951154f22d1fb5efd0,594fb147712d1bf639de86532ff58434,35317b9b780d7f5843d00a207b13c092,d0b88c4ea04cca9bf9835f85df44ed32,2,2,91cf2a0dbffb2e7f14c03e72e704d3e5,a3ac6e509c62ac3d7796c8d8b75f72b2,right,left,unknown,4.68,hmm,337110,"LINESTRING (-121.9120067 38.0142604, -121.9118...",D:/MTC/data/interim/tomtom\bike_rules\1_tomtom...
2,9de152e522f71ed2e8fb4658213e2ad0,2eba19c9a54083c15f8fd70e025559b4,594fb147712d1bf639de86532ff58434,ce8aa7e55b3c5f951154f22d1fb5efd0,88acc59b5faa7dc7ab80892531d77978,d0b88c4ea04cca9bf9835f85df44ed32,2,1,a3ac6e509c62ac3d7796c8d8b75f72b2,91cf2a0dbffb2e7f14c03e72e704d3e5,right,left,unknown,4.68,hmm,337110,LINESTRING (-121.9118829091503 38.014152852369...,D:/MTC/data/interim/tomtom\bike_rules\1_tomtom...
3,ec573127f992ae2dd0fabaa6c6c70cbd,52a5e0b4800e7283accda73c74108762,ce8aa7e55b3c5f951154f22d1fb5efd0,8711d153fefd546c2845baeb0f67bd68,88acc59b5faa7dc7ab80892531d77978,d0b88c4ea04cca9bf9835f85df44ed32,2,2,a3ac6e509c62ac3d7796c8d8b75f72b2,91cf2a0dbffb2e7f14c03e72e704d3e5,right,left,unknown,4.68,hmm,337110,"LINESTRING (-121.9120067 38.0142604, -121.9121...",D:/MTC/data/interim/tomtom\bike_rules\1_tomtom...
4,b09838f5bba85e6f72a5d025c64f4c5f,cb0c7c67acc6c5ae4014a5e6a440e656,06b6e95d0de1a687c0aa3682a8c781b5,3c51d6bd57608d58ba09082b6b5ff904,dcaf838ca94ea539373fbe7d1a3d0e5d,2123da5e29f4836a23922e3725c206cd,2,1,fb2e2b5cf4f18ea282582ba908b35903,69df7691003f0dca1bf9acce7beb9030,left,right,unknown,7.87,hmm,489237,LINESTRING (-121.8990992349619 37.765437480739...,D:/MTC/data/interim/tomtom\bike_rules\1_tomtom...
5,79f5be81b9431c9b6ee5f3a861adf5b2,ffad91c6a5c5eac1803f14c6626da50d,3c51d6bd57608d58ba09082b6b5ff904,0eafc161d1bf63d09e8d17a11d46ee51,dcaf838ca94ea539373fbe7d1a3d0e5d,2123da5e29f4836a23922e3725c206cd,2,2,fb2e2b5cf4f18ea282582ba908b35903,69df7691003f0dca1bf9acce7beb9030,left,right,unknown,7.87,hmm,489237,"LINESTRING (-121.8991717 37.7655042, -121.8991...",D:/MTC/data/interim/tomtom\bike_rules\1_tomtom...
6,031396b53bd320bfa96fd88a8017985d,ffad91c6a5c5eac1803f14c6626da50d,0eafc161d1bf63d09e8d17a11d46ee51,3c51d6bd57608d58ba09082b6b5ff904,fa9f72d6a66c3a9aba647d8fe8c3b305,2123da5e29f4836a23922e3725c206cd,2,1,69df7691003f0dca1bf9acce7beb9030,fb2e2b5cf4f18ea282582ba908b35903,right,right,right,7.87,hmm,489237,"LINESTRING (-121.8992885 37.7656058, -121.8992...",D:/MTC/data/interim/tomtom\bike_rules\1_tomtom...
7,37f3e865c333e5a9a2390ee84d1eaaf7,cb0c7c67acc6c5ae4014a5e6a440e656,3c51d6bd57608d58ba09082b6b5ff904,06b6e95d0de1a687c0aa3682a8c781b5,fa9f72d6a66c3a9aba647d8fe8c3b305,2123da5e29f4836a23922e3725c206cd,2,2,69df7691003f0dca1bf9acce7beb9030,fb2e2b5cf4f18ea282582ba908b35903,right,right,right,7.87,hmm,489237,"LINESTRING (-121.8991717 37.7655042, -121.8991...",D:/MTC/data/interim/tomtom\bike_rules\1_tomtom...
8,456ba758c99bed5ceb9030e44ab36656,1c78571c094618c6be5512ab0ba28e69,c03538f1d44a953787e805d7153c538e,e8661658df8e8f57dfd4be76a73675e6,e9ec16de687222564740bb0f52369866,ecd5b61c731332f50dfbd3c8fe5446cd,1,1,1e21575f9699b6137e985d110b229e0b,bf07674675b8d0a0663694fde88e5e92,right,right,right,6.47,hmm,484729,LINESTRING (-122.4380561316604 38.187805173017...,D:/MTC/data/interim/tomtom\bike_rules\2_tomtom...
9,5541e1cbc5e7f96a016cc90d83d4bf7b,1c78571c094618c6be5512ab0ba28e69,e8661658df8

In [14]:
tomtom_raw_gdf

,ID,FEATTYP,FT,F_JNCTID,F_JNCTTYP,T_JNCTID,T_JNCTTYP,PJ,METERS,FRC,...,TRANS,DYNSPEED,SPEEDCAT,NTHRUTRAF,ROUGHRD,PARTSTRUC,ORDER08,Shape_Length,geometry,tomtom_link_id
0,6.840000e+13,4110,0,6.840001e+13,0,6.840001e+13,0,0,1554.62,7,...,0,0,7,0,0,0,CA097,1989.817478,(LINESTRING (-122.8315739998535 38.62133700023...,1
1,6.840000e+13,4110,0,6.840001e+13,0,6.840001e+13,0,0,551.09,7,...,0,0,6,0,0,0,CA097,704.628862,(LINESTRING (-122.83972099993 38.6229560004714...,2
2,6.840000e+13,4110,0,6.840001e+13,0,6.840002e+13,0,0,53.22,7,...,0,0,7,0,0,0,CA097,68.054041,(LINESTRING (-122.868480999434 38.626759999424...,3
3,6.840000e+13,4110,0,6.840001e+13,0,6.840001e+13,0,0,45.20,7,...,0,0,7,0,0,0,CA097,58.007552,(LINESTRING (-122.8674820000745 38.63032300025...,4
4,6.840000e+13,4110,0,6.840001e+13,0,6.840001e+13,0,0,30.24,7,...,0,0,7,0,0,0,CA097,38.659695,(LINESTRING (-122.8610680001998 38.64033899956...,5
5,6.840000e+13,4110,0,6.840002e+13,0,6.840001e+13,0,0,602.67,6,...,0,0,6,0,0,0,CA097,772.686674,(LINESTRING (-122.9378580002176 38.65743100040...,6
6,6.840000e+13,4110,0,6.840001e+13,0,6.840001e+13,0,0,152.51,7,...,0,0,7,0,0,0,CA097,195.565558,(LINESTRING (-122.9092719998926 38.70904799989...,7
7,6.840000e+13,4110,0,6.840001e+13,0,6.840002e+13,0,0,466.16,7,...,0,0,6,0,0,0,CA097,598.825857,(LINESTRING (-122.9685940004959 38.77782800021...,8
8,6.840000e+13,4110,0,6.840001e+13,0,6.840001e+13,0,0,80.25,6,...,0,0,7,0,0,0,CA097,102.906186,(LINESTRING (-123.0185530001317 38.78901899997...,9
9,6.840000e+13,4110,0,6.840001e+13,0,6.840001e+13,0,0,127.11,7,...,0,0,7,0,0,0,CA097,162.910525,(LINESTRING (-123.0254980004591 38.79289899992...,10


In [15]:
tomtom_raw_gdf.TOLLRD.value_counts()

0     601466
11       480
21       114
Name: TOLLRD, dtype: int64

In [16]:
print(tomtom_raw_gdf.shape)
print(tomtom_match_gdf.shape)

print(tomtom_raw_gdf.columns)
print(tomtom_match_gdf.columns)

(602060, 54)
(1749759, 18)
Index(['ID', 'FEATTYP', 'FT', 'F_JNCTID', 'F_JNCTTYP', 'T_JNCTID', 'T_JNCTTYP',
       'PJ', 'METERS', 'FRC', 'NETCLASS', 'NETBCLASS', 'NET2CLASS', 'NAME',
       'NAMELC', 'SOL', 'NAMETYP', 'CHARGE', 'SHIELDNUM', 'RTETYP', 'RTEDIR',
       'RTEDIRVD', 'PROCSTAT', 'FOW', 'SLIPRD', 'FREEWAY', 'BACKRD', 'TOLLRD',
       'RDCOND', 'STUBBLE', 'PRIVATERD', 'CONSTATUS', 'ONEWAY', 'F_BP', 'T_BP',
       'F_ELEV', 'T_ELEV', 'KPH', 'MINUTES', 'POSACCUR', 'CARRIAGE', 'LANES',
       'RAMP', 'ADA', 'TRANS', 'DYNSPEED', 'SPEEDCAT', 'NTHRUTRAF', 'ROUGHRD',
       'PARTSTRUC', 'ORDER08', 'Shape_Length', 'geometry', 'tomtom_link_id'],
      dtype='object')
Index(['shstReferenceId', 'shstGeometryId', 'fromIntersectionId',
       'toIntersectionId', 'gisReferenceId', 'gisGeometryId',
       'gisTotalSegments', 'gisSegmentIndex', 'gisFromIntersectionId',
       'gisToIntersectionId', 'startSideOfStreet', 'endSideOfStreet',
       'sideOfStreet', 'score', 'matchType', 'pp_tomto

In [17]:
# NAME, SHEILDNUM, RTEDIR
tomtom_raw_gdf.FREEWAY.value_counts()

0    586165
1     15895
Name: FREEWAY, dtype: int64

In [18]:
tomtom_match_gdf.pp_tomtom_link_id.max()
print("Sharedstreets matched "+ str(tomtom_match_gdf.pp_tomtom_link_id.nunique()) +" out of " + str(len(tomtom_raw_gdf)) 
      + " total tomtom links.")

Sharedstreets matched 550620 out of 602060 total tomtom links.


In [19]:
tomtom_raw_gdf["tomtom_link_id"] = range(1, len(tomtom_raw_gdf)+1)

tomtom_gdf = pd.merge(tomtom_match_gdf, 
                      tomtom_raw_gdf[['tomtom_link_id', 'ID', 'F_JNCTID', 'T_JNCTID', 'LANES', "FRC", "NAME", "SHIELDNUM",
                                      "RTEDIR"]],
                     how = "left",
                     left_on = ['pp_tomtom_link_id'],
                     right_on = ['tomtom_link_id']
                     )
                                 
print(tomtom_gdf.shape)

(1749759, 27)


# Join network with TomTom

In [102]:
# join tomtom with network

unique_tomtom_match_gdf = tomtom_gdf.drop_duplicates(
    subset = ['shstReferenceId', 'shstGeometryId', 'fromIntersectionId', 'toIntersectionId']).copy()

unique_tomtom_match_gdf.rename(columns = {"ID" : "tomtom_ID", "LANES" : "tomtom_lanes", "FRC" : "tomtom_FRC",
                                          "NAME" : "tomtom_name", "SHIELDNUM" : "tomtom_shieldnum", 
                                          "RTEDIR" : "tomtom_rtedir"},
                              inplace = True)

link_with_tomtom_gdf = pd.merge(link_gdf,
                            unique_tomtom_match_gdf[['shstReferenceId', 'shstGeometryId', 'fromIntersectionId','toIntersectionId',
                                                     'pp_tomtom_link_id', "tomtom_ID", 'F_JNCTID', 'T_JNCTID', 
                                                     "tomtom_lanes", "tomtom_FRC", "tomtom_name", "tomtom_shieldnum",
                                                     "tomtom_rtedir"]],
                                  how = "left",
                                  on = ['shstReferenceId', 'shstGeometryId', 'fromIntersectionId', 'toIntersectionId'],
                                  )

In [21]:
link_with_tomtom_gdf[link_with_tomtom_gdf.shstGeometryId.isnull()]
link_with_tomtom_gdf[~(link_with_tomtom_gdf.shstGeometryId == link_with_tomtom_gdf.id)]
link_with_tomtom_gdf.lanes.value_counts()

nan                                  1359907
                                      134553
2                                      84235
[nan, nan]                             46733
3                                      13260
4                                      11417
[nan, nan, nan]                         9469
1                                       8812
5                                       4697
[, ]                                    4198
[2, 3]                                  2697
[2, 2]                                  1977
[nan, ]                                 1784
[, nan]                                 1177
[3, 3]                                  1043
[3, 4]                                   977
[nan, nan, nan, nan]                     960
[, , ]                                   960
[2, 2, 2]                                767
6                                        725
[2, nan]                                 723
[4, 4]                                   686
[3, 2]    

In [104]:
link_with_tomtom_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 1705772 entries, 0 to 1705771
Data columns (total 40 columns):
access                object
area                  object
bike_access           int64
bridge                object
drive_access          int64
est_width             object
fromIntersectionId    object
highway               object
id                    object
junction              object
key                   object
landuse               object
lanes                 object
link                  object
maxspeed              object
name                  object
oneWay                object
ref                   object
roadway               object
roundabout            object
service               object
shstGeometryId        object
shstReferenceId       object
toIntersectionId      object
tunnel                object
u                     int64
v                     int64
walk_access           int64
wayId                 object
width                 object
geometry      

In [105]:
link_with_tomtom_gdf.u.nunique()

661023

In [106]:
link_with_tomtom_gdf.fromIntersectionId.nunique()

661023

In [107]:
len(set(link_with_tomtom_gdf.u.tolist() + link_with_tomtom_gdf.v.tolist()))

661159

# Read thrid party shst match results

In [26]:
# read TM2 non Marin conflation result

tm2nonMarin_match_gdf = read_shst_extract(data_interim_dir + "tm2_nonMarin/", "*tm2nonMarin.out.matched.geojson")

tm2nonMarin_match_gdf.rename(columns = {"shstFromIntersectionId" : "fromIntersectionId",
                                   "shstToIntersectionId" : "toIntersectionId"},
                       inplace = True)


----------start reading shst extraction data-------------
reading shst extraction data :  ../../data/interim/tm2_nonMarin\car_rules\10_tm2nonMarin.out.matched.geojson
reading shst extraction data :  ../../data/interim/tm2_nonMarin\car_rules\11_tm2nonMarin.out.matched.geojson
reading shst extraction data :  ../../data/interim/tm2_nonMarin\car_rules\12_tm2nonMarin.out.matched.geojson
reading shst extraction data :  ../../data/interim/tm2_nonMarin\car_rules\13_tm2nonMarin.out.matched.geojson
reading shst extraction data :  ../../data/interim/tm2_nonMarin\car_rules\14_tm2nonMarin.out.matched.geojson
reading shst extraction data :  ../../data/interim/tm2_nonMarin\car_rules\1_tm2nonMarin.out.matched.geojson
reading shst extraction data :  ../../data/interim/tm2_nonMarin\car_rules\2_tm2nonMarin.out.matched.geojson
reading shst extraction data :  ../../data/interim/tm2_nonMarin\car_rules\3_tm2nonMarin.out.matched.geojson
reading shst extraction data :  ../../data/interim/tm2_nonMarin\car_rules

In [27]:
# read TM2 Marin conflation result

tm2marin_match_gdf = read_shst_extract(data_interim_dir + "tm2_Marin/", "*tm2Marin.out.matched.geojson")

tm2marin_match_gdf.rename(columns = {"shstFromIntersectionId" : "fromIntersectionId",
                                   "shstToIntersectionId" : "toIntersectionId"},
                       inplace = True)

----------start reading shst extraction data-------------
reading shst extraction data :  ../../data/interim/tm2_Marin\car_rules\10_tm2Marin.out.matched.geojson
reading shst extraction data :  ../../data/interim/tm2_Marin\car_rules\11_tm2Marin.out.matched.geojson
reading shst extraction data :  ../../data/interim/tm2_Marin\car_rules\12_tm2Marin.out.matched.geojson
reading shst extraction data :  ../../data/interim/tm2_Marin\car_rules\13_tm2Marin.out.matched.geojson
reading shst extraction data :  ../../data/interim/tm2_Marin\car_rules\14_tm2Marin.out.matched.geojson
reading shst extraction data :  ../../data/interim/tm2_Marin\car_rules\1_tm2Marin.out.matched.geojson
reading shst extraction data :  ../../data/interim/tm2_Marin\car_rules\2_tm2Marin.out.matched.geojson
reading shst extraction data :  ../../data/interim/tm2_Marin\car_rules\3_tm2Marin.out.matched.geojson
reading shst extraction data :  ../../data/interim/tm2_Marin\car_rules\4_tm2Marin.out.matched.geojson
reading shst extrac

In [28]:
# read sfcta true shape conflation result

sfcta_match_gdf = read_shst_extract(data_interim_dir + "stclines/", "*sfcta.out.matched.geojson")

sfcta_match_gdf.rename(columns = {"shstFromIntersectionId" : "fromIntersectionId",
                                   "shstToIntersectionId" : "toIntersectionId"},
                       inplace = True)

----------start reading shst extraction data-------------
reading shst extraction data :  ../../data/interim/stclines\car_rules\sfcta.out.matched.geojson
reading shst extraction data :  ../../data/interim/stclines\ped_rules\sfcta.out.matched.geojson
----------finished reading shst extraction data-------------


In [29]:
# read sfcta stick network conflation result

sfcta_stick_match_gdf = read_shst_extract(data_interim_dir + "sfcta/", "*sfcta.out.matched.geojson")

sfcta_stick_match_gdf.rename(columns = {"shstFromIntersectionId" : "fromIntersectionId",
                                   "shstToIntersectionId" : "toIntersectionId"},
                       inplace = True)

----------start reading shst extraction data-------------
reading shst extraction data :  ../../data/interim/sfcta\car_rules\sfcta.out.matched.geojson
reading shst extraction data :  ../../data/interim/sfcta\ped_rules\sfcta.out.matched.geojson
reading shst extraction data :  ../../data/interim/sfcta\reverse_dir\sfcta.out.matched.geojson
----------finished reading shst extraction data-------------


In [30]:
# read pems conflation result

pems_match_gdf = gpd.read_file(data_interim_dir + "mtc/pems_conflation_result.geojson")

In [31]:
tm2nonMarin_match_gdf.shstReferenceId.nunique()

758537

In [32]:
tm2nonMarin_match_gdf.shstReferenceId.nunique()

758537

In [33]:
tm2marin_match_gdf.shstReferenceId.nunique()

758534

In [34]:
tm2marin_match_gdf.shstReferenceId.nunique()

758534

In [35]:
sfcta_match_gdf.shstReferenceId.nunique()

37449

In [36]:
sfcta_stick_match_gdf.shstReferenceId.nunique()

34910

In [37]:
pems_match_gdf[pems_match_gdf.station == 401943]

,station,district,route,direction,type,time_period,lanes,median_flow,avg_flow,sd_flow,...,abs_pm,latitude,longitude,year,shstReferenceId,tomtom_shieldnum,tomtom_rtedir,snap_distance,source,geometry
33050,401943,4,280,S,ML,AM,4,15090.5,14497.000000,1815.697442,...,2.85,37.322361,-121.8978,2014,005586c0c986e59d4dcd979513f7e8e6,280,S,4.06128,nearest,POINT (-2086117.780704529 4546249.146790181)
33051,401943,4,280,S,ML,EA,4,1129.0,1130.241379,59.193180,...,2.85,37.322361,-121.8978,2014,005586c0c986e59d4dcd979513f7e8e6,280,S,4.06128,nearest,POINT (-2086117.780704529 4546249.146790181)
33052,401943,4,280,S,ML,EV,4,17841.5,17727.236842,1127.814879,...,2.85,37.322361,-121.8978,2014,005586c0c986e59d4dcd979513f7e8e6,280,S,4.06128,nearest,POINT (-2086117.780704529 4546249.146790181)
33053,401943,4,280,S,ML,MD,4,21750.5,21328.181818,3004.852264,...,2.85,37.322361,-121.8978,2014,005586c0c986e59d4dcd979513f7e8e6,280,S,4.06128,nearest,POINT (-2086117.780704529 4546249.146790181)
33054,401943,4,280,S,ML,PM,4,25372.0,24350.500000,3805.566827,...,2.85,37.322361,-121.8978,2014,005586c0c986e59d4dcd979513f7e8e6,280,S,4.06128,nearest,POINT (-2086117.780704529 4546249.146790181)
40307,401943,4,280,S,ML,AM,4,14900.5,14798.814815,839.782880,...,2.85,37.322361,-121.8978,2015,005586c0c986e59d4dcd979513f7e8e6,280,S,4.06128,nearest,POINT (-2086117.780704529 4546249.146790181)
40308,401943,4,280,S,ML,EA,4,1141.0,1146.937500,49.355164,...,2.85,37.322361,-121.8978,2015,005586c0c986e59d4dcd979513f7e8e6,280,S,4.06128,nearest,POINT (-2086117.780704529 4546249.146790181)
40309,401943,4,280,S,ML,EV,4,17769.5,17732.175000,716.354764,...,2.85,37.322361,-121.8978,2015,005586c0c986e59d4dcd979513f7e8e6,280,S,4.06128,nearest,POINT (-2086117.780704529 4546249.146790181)
40310,401943,4,280,S,ML,MD,4,21372.0,21209.777778,976.877630,...,2.85,37.322361,-121.8978,2015,005586c0c986e59d4dcd979513f7e8e6,280,S,4.06128,nearest,POINT (-2086117.780704529 4546249.146790181)
40311,401943,4,280,S,ML,PM,4,23156.0,22938.693878,886.817089,...,2.85,37.322361,-121.8978,2015,005586c0c986e59d4dcd979513f7e8e6,280,S,4.06128,nearest,POINT (-2086117.780704529 4546249.146790181)


In [38]:
pems_match_gdf.columns

Index(['station', 'district', 'route', 'direction', 'type', 'time_period',
       'lanes', 'median_flow', 'avg_flow', 'sd_flow', 'median_speed',
       'avg_speed', 'sd_speed', 'median_occup', 'avg_occup', 'sd_occupancy',
       'days_observed', 'state_pm', 'abs_pm', 'latitude', 'longitude', 'year',
       'shstReferenceId', 'tomtom_shieldnum', 'tomtom_rtedir', 'snap_distance',
       'source', 'geometry'],
      dtype='object')

In [39]:
pems_match_gdf['type'].value_counts()

ML    60319
OR     9309
FR     5733
FF      666
HV       30
Name: type, dtype: int64

In [40]:
# link can have multiple pems station on it, so trying to get the mode of #lanes by station type

pems_lanes_df = pems_match_gdf[(pems_match_gdf.year.isin([2014,2015,2016]))].groupby(
    ["shstReferenceId", "type", "lanes"]
)["station"].count().sort_values(ascending = False).reset_index().drop_duplicates(subset = ['shstReferenceId', 'type'])

pems_lanes_df = pems_lanes_df.pivot_table(index = ["shstReferenceId"], values = "lanes", columns = "type").fillna(0).reset_index()

In [41]:
link_pems_df = pems_match_gdf[(pems_match_gdf.year.isin([2014,2015,2016]))].drop_duplicates(
    subset = ["shstReferenceId", "station"]).groupby(
    ["shstReferenceId"]
)["station"].apply(list).reset_index().rename(columns = {"station" : "PEMSID"})

pems_lanes_df = pd.merge(
    pems_lanes_df,
    link_pems_df,
    how = "left",
    on = "shstReferenceId"
)

In [42]:
link_pems_df

,shstReferenceId,PEMSID
0,000619909cfedfdc3ae846759247e09f,"[404433, 407341]"
1,001faa1f8bf0fafb4298b7438a83b506,[400615]
2,00441ada4f77ea0e20dd17acd688e289,[404649]
3,005586c0c986e59d4dcd979513f7e8e6,"[401943, 402067, 403410, 403411, 403412]"
4,005a078054386cb7659bcbbdacf61c63,[400929]
5,00ff1d8493ea56e766bedbd47663cbb0,"[407358, 407359]"
6,01248bbf4c897c845921a9c93c8d609d,"[407328, 407329, 407330]"
7,0167c71940194a800e0c35023a9933c6,"[409722, 409723]"
8,018c32728b73695ac776d4181f35ecf6,[401134]
9,01a23a9cd090dd752fe36221a58a9b92,"[406606, 406605]"


In [43]:
pems_lanes_df

,shstReferenceId,FF,FR,HV,ML,OR,PEMSID
0,000619909cfedfdc3ae846759247e09f,0.0,0.0,0.0,3.0,0.0,"[404433, 407341]"
1,001faa1f8bf0fafb4298b7438a83b506,0.0,0.0,0.0,3.0,0.0,[400615]
2,00441ada4f77ea0e20dd17acd688e289,0.0,0.0,0.0,3.0,0.0,[404649]
3,005586c0c986e59d4dcd979513f7e8e6,0.0,1.0,0.0,4.0,1.0,"[401943, 402067, 403410, 403411, 403412]"
4,005a078054386cb7659bcbbdacf61c63,0.0,0.0,0.0,5.0,0.0,[400929]
5,00ff1d8493ea56e766bedbd47663cbb0,2.0,0.0,1.0,0.0,0.0,"[407358, 407359]"
6,01248bbf4c897c845921a9c93c8d609d,0.0,1.0,0.0,3.0,1.0,"[407328, 407329, 407330]"
7,0167c71940194a800e0c35023a9933c6,0.0,0.0,0.0,3.0,1.0,"[409722, 409723]"
8,018c32728b73695ac776d4181f35ecf6,0.0,0.0,0.0,3.0,0.0,[401134]
9,01a23a9cd090dd752fe36221a58a9b92,0.0,1.0,0.0,4.0,0.0,"[406606, 406605]"


In [76]:
# join match result with source data

# TM2 non Marin
unique_tm2nonMarin_match_gdf = tm2nonMarin_match_gdf.drop_duplicates(
    subset = ['shstReferenceId', 'shstGeometryId', 'fromIntersectionId', 'toIntersectionId']).copy()

unique_tm2nonMarin_match_gdf = pd.merge(unique_tm2nonMarin_match_gdf, 
                      tm2_link_roadway_gdf[['A', 'B', "NUMLANES", "FT", "ASSIGNABLE"]],
                     how = "left",
                     left_on = ['pp_a', "pp_b"],
                     right_on = ['A', "B"]
                     )

unique_tm2nonMarin_match_gdf.rename(columns = {"A" : "TM2_A", "B" : "TM2_B", "NUMLANES" : "TM2_LANES", 
                                               "FT" : "TM2_FT", "ASSIGNABLE" : "TM2_ASSIGNABLE"},
                                   inplace = True)

# TM2 Marin

unique_tm2marin_match_gdf = tm2marin_match_gdf.drop_duplicates(
    subset = ['shstReferenceId', 'shstGeometryId', 'fromIntersectionId', 'toIntersectionId']).copy()

unique_tm2marin_match_gdf = pd.merge(unique_tm2marin_match_gdf, 
                      tm2_marin_link_roadway_gdf[['A', 'B', "NUMLANES", "FT", "ASSIGNABLE"]],
                     how = "left",
                     left_on = ['pp_a', "pp_b"],
                     right_on = ['A', "B"]
                     )

unique_tm2marin_match_gdf.rename(columns = {"A" : "TM2Marin_A", "B" : "TM2Marin_B", "NUMLANES" : "TM2Marin_LANES", 
                                               "FT" : "TM2Marin_FT", "ASSIGNABLE" : "TM2Marin_ASSIGNABLE"},
                                   inplace = True)

# sfcta stick and true shape
unique_sfcta_match_gdf = sfcta_stick_match_gdf.drop_duplicates(
    subset = ['shstReferenceId', 'shstGeometryId', 'fromIntersectionId', 'toIntersectionId']).copy()

"""
# sfcta true shape
unique_sfcta_match_gdf = pd.merge(unique_sfcta_match_gdf, 
                      sfcta_gdf[['sfcta_trueshape_id', "A", "B", "AB"]],
                     how = "left",
                     left_on = ['pp_sfcta_trueshape_id'],
                     right_on = ['sfcta_trueshape_id']
                     )

sfcta_stick_gdf = gpd.read_file("../../data/external/sfcta/SanFrancisco_links.shp")
"""
unique_sfcta_match_gdf = pd.merge(unique_sfcta_match_gdf,
                                  sfcta_stick_gdf[["A", "B", "FT", "STREETNAME", "LANE_AM", "LANE_OP", "LANE_PM"]],
                                  how = "left",
                                  left_on = ["pp_a", "pp_b"],
                                  right_on = ["A", "B"])

unique_sfcta_match_gdf.rename(columns = {"A" : "sfcta_A", "B" : "sfcta_B", "FT" : "sfcta_FT", "STREETNAME" : "sfcta_STREETNAME",
                                         "LANE_AM" : "sfcta_LANE_AM", "LANE_OP" : "sfcta_LANE_OP", "LANE_PM" : "sfcta_LANE_PM"},
                             inplace = True)

# pems
pems_lanes_df.rename(columns = {"FF" : "pems_lanes_FF", "FR" : "pems_lanes_FR", "HV" : "pems_lanes_HV", 
                                "ML" : "pems_lanes_ML", "OR" : "pems_lanes_OR"},
                     inplace = True)

In [77]:
pems_lanes_df.columns

Index(['shstReferenceId', 'pems_lanes_FF', 'pems_lanes_FR', 'pems_lanes_HV',
       'pems_lanes_ML', 'pems_lanes_OR', 'PEMSID'],
      dtype='object')

In [78]:
unique_sfcta_match_gdf.shstReferenceId.shape

(34910,)

In [87]:
link_all_conflated_gdf.sfcta_LANE_AM.notnull().sum()

34906

# Join network with third party sources

In [108]:
link_all_conflated_gdf = pd.merge(
    link_with_tomtom_gdf,
    unique_tm2nonMarin_match_gdf[['shstReferenceId', 'shstGeometryId', 'fromIntersectionId',
       'toIntersectionId', 'TM2_A', 'TM2_B', 'TM2_LANES', 'TM2_FT', 'TM2_ASSIGNABLE']],
    how = "left",
    on = ['shstReferenceId', 'shstGeometryId', 'fromIntersectionId', "toIntersectionId"]
)

In [109]:
link_all_conflated_gdf = pd.merge(
    link_all_conflated_gdf,
    unique_tm2marin_match_gdf[['shstReferenceId', 'shstGeometryId', 'fromIntersectionId',
       'toIntersectionId', 'TM2Marin_A', 'TM2Marin_B', 'TM2Marin_LANES', 'TM2Marin_FT',
       'TM2Marin_ASSIGNABLE']],
    how = "left",
    on = ['shstReferenceId', 'shstGeometryId', 'fromIntersectionId', "toIntersectionId"]
)

In [110]:
link_all_conflated_gdf = pd.merge(
    link_all_conflated_gdf,
    unique_sfcta_match_gdf[['shstReferenceId', 'shstGeometryId', 'fromIntersectionId',
       'toIntersectionId', 'sfcta_A', 'sfcta_B', 'sfcta_FT', 'sfcta_STREETNAME', 'sfcta_LANE_AM', 'sfcta_LANE_OP',
       'sfcta_LANE_PM']],
    how = "left",
    on = ['shstReferenceId', 'shstGeometryId', 'fromIntersectionId', "toIntersectionId"]
)

In [111]:
link_all_conflated_gdf = pd.merge(
    link_all_conflated_gdf,
    pems_lanes_df,
    how = "left",
    on = "shstReferenceId"
)

In [112]:
link_all_conflated_gdf.shape

(1705772, 63)

In [113]:
link_all_conflated_gdf.columns

Index(['access', 'area', 'bike_access', 'bridge', 'drive_access', 'est_width',
       'fromIntersectionId', 'highway', 'id', 'junction', 'key', 'landuse',
       'lanes', 'link', 'maxspeed', 'name', 'oneWay', 'ref', 'roadway',
       'roundabout', 'service', 'shstGeometryId', 'shstReferenceId',
       'toIntersectionId', 'tunnel', 'u', 'v', 'walk_access', 'wayId', 'width',
       'geometry', 'pp_tomtom_link_id', 'tomtom_ID', 'F_JNCTID', 'T_JNCTID',
       'tomtom_lanes', 'tomtom_FRC', 'tomtom_name', 'tomtom_shieldnum',
       'tomtom_rtedir', 'TM2_A', 'TM2_B', 'TM2_LANES', 'TM2_FT',
       'TM2_ASSIGNABLE', 'TM2Marin_A', 'TM2Marin_B', 'TM2Marin_LANES',
       'TM2Marin_FT', 'TM2Marin_ASSIGNABLE', 'sfcta_A', 'sfcta_B', 'sfcta_FT',
       'sfcta_STREETNAME', 'sfcta_LANE_AM', 'sfcta_LANE_OP', 'sfcta_LANE_PM',
       'pems_lanes_FF', 'pems_lanes_FR', 'pems_lanes_HV', 'pems_lanes_ML',
       'pems_lanes_OR', 'PEMSID'],
      dtype='object')

In [114]:
link_all_conflated_gdf.head(30).lanes.iloc[19][1]

'2'

# Write out standard link json and link feather

In [88]:
link_json_columns = link_df.columns.tolist()

In [89]:
link_json_columns

['access',
 'area',
 'bike_access',
 'bridge',
 'drive_access',
 'est_width',
 'fromIntersectionId',
 'highway',
 'id',
 'junction',
 'key',
 'landuse',
 'lanes',
 'link',
 'maxspeed',
 'name',
 'oneWay',
 'ref',
 'roadway',
 'roundabout',
 'service',
 'shstGeometryId',
 'shstReferenceId',
 'toIntersectionId',
 'tunnel',
 'u',
 'v',
 'walk_access',
 'wayId',
 'width']

In [119]:
link_all_conflated_gdf.columns

Index(['access', 'area', 'bike_access', 'bridge', 'drive_access', 'est_width',
       'fromIntersectionId', 'highway', 'id', 'junction', 'key', 'landuse',
       'lanes', 'link', 'maxspeed', 'name', 'oneWay', 'ref', 'roadway',
       'roundabout', 'service', 'shstGeometryId', 'shstReferenceId',
       'toIntersectionId', 'tunnel', 'u', 'v', 'walk_access', 'wayId', 'width',
       'geometry', 'pp_tomtom_link_id', 'tomtom_ID', 'F_JNCTID', 'T_JNCTID',
       'tomtom_lanes', 'tomtom_FRC', 'tomtom_name', 'tomtom_shieldnum',
       'tomtom_rtedir', 'TM2_A', 'TM2_B', 'TM2_LANES', 'TM2_FT',
       'TM2_ASSIGNABLE', 'TM2Marin_A', 'TM2Marin_B', 'TM2Marin_LANES',
       'TM2Marin_FT', 'TM2Marin_ASSIGNABLE', 'sfcta_A', 'sfcta_B', 'sfcta_FT',
       'sfcta_STREETNAME', 'sfcta_LANE_AM', 'sfcta_LANE_OP', 'sfcta_LANE_PM',
       'pems_lanes_FF', 'pems_lanes_FR', 'pems_lanes_HV', 'pems_lanes_ML',
       'pems_lanes_OR', 'PEMSID', 'tomtom_FRC_def', 'TM2_FT_def'],
      dtype='object')

In [115]:
link_all_conflated_gdf[link_all_conflated_gdf.shstReferenceId == "6233a20ff0d9dc29be597a06ffd19727"]

,access,area,bike_access,bridge,drive_access,est_width,fromIntersectionId,highway,id,junction,...,sfcta_STREETNAME,sfcta_LANE_AM,sfcta_LANE_OP,sfcta_LANE_PM,pems_lanes_FF,pems_lanes_FR,pems_lanes_HV,pems_lanes_ML,pems_lanes_OR,PEMSID
348788,nan,nan,1,nan,0,nan,f407efc156163c556b516200a966187e,track,b5a13d71332300baf271a3dde50dd06d,nan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [116]:
object_col = list(link_all_conflated_gdf.select_dtypes(['object']).columns)

In [117]:
print("-------write out link json---------")

#link_prop = link_all_conflated_gdf.drop("geometry", axis = 1).columns.tolist()
link_prop = link_json_columns

out = link_all_conflated_gdf[link_prop].to_json(orient = "records")

with open(data_interim_dir + 'step4_conflate_with_tomtom/link.json', 'w') as f:
    f.write(out)

-------write out link json---------


In [118]:
print("-------write out link feather---------")

link_feather = link_all_conflated_gdf[link_json_columns].copy()

object_col = list(link_feather.select_dtypes(['object']).columns)

for c in object_col:
    link_feather[c] = link_feather[c].astype(str)

link_feather.to_feather(data_interim_dir + 'step4_conflate_with_tomtom/link.feather')

-------write out link feather---------


In [119]:
link_all_conflated_gdf_prop = link_all_conflated_gdf.drop("geometry", axis = 1).columns
link_all_conflated_geojson = link_df_to_geojson(link_all_conflated_gdf, link_all_conflated_gdf_prop)

with open(data_interim_dir + 'step4_conflate_with_tomtom/link.geojson', "w") as f:
    json.dump(link_all_conflated_geojson, f)

# Write out conflation result data base

In [120]:
# tomtom FRC dictionary

tomtom_FRC_dict = {
    0: "0-Motorway, Freeway, or Other Major Road",
    1: "1-Major Road Less Important than a Motorway",
    2: "2-Other Major Road",
    3: "3-Secondary Road",
    4: "4-Local Connecting Road",
    5: "5-Local Road of High Importance",
    6: "6-Local Road",
    7: "7-Local Road of Minor Importance",
    8: "8-Other Road"
}

link_all_conflated_gdf["tomtom_FRC_def"] = link_all_conflated_gdf["tomtom_FRC"].map(tomtom_FRC_dict)

In [121]:
link_all_conflated_gdf.tomtom_FRC_def.value_counts()

6-Local Road                                   345262
7-Local Road of Minor Importance               325150
5-Local Road of High Importance                 83747
4-Local Connecting Road                         71881
8-Other Road                                    35400
3-Secondary Road                                14483
1-Major Road Less Important than a Motorway      1851
0-Motorway, Freeway, or Other Major Road         1217
2-Other Major Road                                408
Name: tomtom_FRC_def, dtype: int64

In [122]:
TM2_FT_dict = {
    0: "0-Connector",
    1: "1-Freeway to Freeway",
    2: "2-Freeway",
    3: "3-Expressway",
    4: "4-Collector",
    5: "5-Ramp",
    6: "6-Special Facility",
    7: "7-Major Arterial",
}

link_all_conflated_gdf["TM2_FT_def"] = link_all_conflated_gdf["TM2_FT"].map(TM2_FT_dict)

link_all_conflated_gdf.TM2_FT_def.value_counts()

4-Collector             540491
7-Major Arterial        141340
0-Connector              56409
3-Expressway             13888
2-Freeway                 3271
5-Ramp                    2168
1-Freeway to Freeway       156
Name: TM2_FT_def, dtype: int64

In [123]:
# write conflation data base

tableau_fields = ["shstReferenceId", "roadway", "lanes", "drive_access", "bike_access", "walk_access",
                 "tomtom_FRC", "tomtom_FRC_def", "tomtom_lanes", "pp_tomtom_link_id", 'F_JNCTID', 'T_JNCTID',
                  'tomtom_name', 'tomtom_shieldnum', 'tomtom_rtedir', 'TM2Marin_A',
                   'TM2Marin_B', 'TM2Marin_FT', 'TM2Marin_LANES', 'TM2Marin_ASSIGNABLE', 
                  'TM2_A', 'TM2_B', 'TM2_FT', "TM2_FT_def", 'TM2_LANES', 'TM2_ASSIGNABLE', 
                  "sfcta_A", 'sfcta_B', "sfcta_STREETNAME", 'sfcta_FT', 'sfcta_LANE_AM', 'sfcta_LANE_OP',
                   'sfcta_LANE_PM', 'PEMSID', 'pems_lanes_FF', 'pems_lanes_FR',
       'pems_lanes_HV', 'pems_lanes_ML', 'pems_lanes_OR']

link_all_conflated_gdf[tableau_fields].rename(columns = {"lanes" : "lanes_osm",
                                                         "pp_tomtom_link_id" : "tomtom_unique_id"}
                              ).to_csv(data_interim_dir + "conflation_result.csv",
                             index = False)

In [124]:
link_all_conflated_gdf[link_all_conflated_gdf.shstReferenceId == "0d2e3cab2e51b61fc6b42641fa648540"].sfcta_LANE_PM

47077    0.0
Name: sfcta_LANE_PM, dtype: float64

In [125]:
pd.crosstab(link_all_conflated_gdf.TM2_ASSIGNABLE, link_all_conflated_gdf.TM2_LANES)

TM2_LANES,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0
TM2_ASSIGNABLE,,,,,,,,,,
0.0,56409,1690,1,0,0,0,0,0,0,0
1.0,0,642776,37448,14234,3813,1151,186,12,2,1


In [126]:
link_all_conflated_gdf[link_all_conflated_gdf.TM2_A.notnull()].shape

(757723, 65)

In [127]:
pd.crosstab(link_all_conflated_gdf.TM2Marin_ASSIGNABLE, link_all_conflated_gdf.TM2Marin_LANES)

TM2Marin_LANES,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0
TM2Marin_ASSIGNABLE,,,,,,,,,,
0.0,56409,1692,1,0,0,0,0,0,0,0
1.0,0,642194,37944,14294,3828,1155,188,12,2,1


In [128]:
link_all_conflated_gdf[link_all_conflated_gdf.TM2Marin_A.notnull()].shape

(757720, 65)

In [129]:
link_all_conflated_gdf[link_all_conflated_gdf.PEMSID.notnull()].shape

(1389, 65)

In [130]:
link_all_conflated_gdf[link_all_conflated_gdf.pp_tomtom_link_id.notnull()].shape

(879399, 65)

In [131]:
link_all_conflated_gdf[link_all_conflated_gdf.sfcta_A.notnull()].shape

(34906, 65)